![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2F05+-+TensorFlow&file=05b+-+Vertex+AI+Custom+Model+-+TensorFlow+-+Custom+Job+With+Python+Source+Distribution.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/05%20-%20TensorFlow/05b%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Custom%20Job%20With%20Python%20Source%20Distribution.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2F05%2520-%2520TensorFlow%2F05b%2520-%2520Vertex%2520AI%2520Custom%2520Model%2520-%2520TensorFlow%2520-%2520Custom%2520Job%2520With%2520Python%2520Source%2520Distribution.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/05%20-%20TensorFlow/05b%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Custom%20Job%20With%20Python%20Source%20Distribution.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/05%20-%20TensorFlow/05b%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Custom%20Job%20With%20Python%20Source%20Distribution.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

---
---
**Changes In Progress**

Every attempt is being made to ensure the public version of this notebook runs without error while the follow enhancements are being made:
- The workflow of the notebook is being adapted to a Kubeflow Pipeline running on Vertex AI Pipelines
- Including Evaluation data within Vertex AI Model Registry
- Update the Vertex AI Experiments integration which has been great simplified within the API over the past few months
- add client library reference links to each section

Order: notebook 05 and 05a will be updated first.  Then 05b-05i will follow quickly.

This note will be removed once these changes are complete.

---
---

# 05b - Vertex AI > Training > Custom Jobs - With Python Source Distribution

**05 Series Overview**

>**NOTE:** The notebooks in the `05 - TensorFlow` series demonstrate training, serving and operations for TensorFlow models and take advantage of [Vertex AI TensorBoard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) to track training across experiments.  Running these notebooks will create a Vertex AI TensorBoard instance which previously (before August 2023) had a subscription cost but is now priced based on storage of which this notebook will create minimal size (<2MB). - [Vertex AI Pricing](https://cloud.google.com/vertex-ai/pricing#tensorboard).

Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the [05 - Vertex AI Custom Model - TensorFlow - in Notebook](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20in%20Notebook.ipynb) notebook, the model training happened directly in the notebook.  The model was then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `05a-05i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI Training as managed jobs.  These jobs will be demonstrated as:

-  [Custom Job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job) that trains and saves (to GCS) a model from a python script (`05a`), python source distribution (`05b`), and custom container (`05c`)
-  [Training Pipeline](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview) that trains and registers a model from a python script (`05d`), python source distribution (`05e`), and custom container (`05f`)
-  [Hyperparameter Tuning Jobs](https://cloud.google.com/vertex-ai/docs/training/create-training-pipeline) from a python script (`05g`), python source distribution (`05h`), and custom container (`05i`)

**This Notebook (`05b`): An extension of `05a` that saves the script within a source distribution**

This notebook trains the same Tensorflow Keras model from [05 - Vertex AI Custom Model - TensorFlow - in Notebook](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20in%20Notebook.ipynb) by first modifying and saving the training code to a Python script as shown in [05 - Vertex AI Custom Model - TensorFlow - Notebook to Script](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Notebook%20to%20Script.ipynb).  

A Python source distribution is built containing the script.  For more guidance on this process visit the tip notebook [Python Packages](../Tips/Python%20Packages.ipynb).  The source distribution is then used as an input for a Vertex AI > Training > Custom Job that is also assigned compute resources and a [pre-built container for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for executing the training in a managed service.  While this example fits nicely in a single script, larger examples will benefit from the flexibility offered by source distributions and this notebook gives an example of making the shift.

This job is launched using the Vertex AI client library:
- [Python Cloud Client Libraries](https://cloud.google.com/python/docs/reference)
    - [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)
        - [`aiplatform` package](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
            - [`aiplatform.CustomJob()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomJob#google_cloud_aiplatform_CustomJob)
                - with [worker_pool_specs](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.WorkerPoolSpec) using `python_package_spec`

|Custom Job|Workflow With Custom Job|
|:---:|:---:|
|![](../architectures/architectures/images/custom%20training/b_job.png)|![](../architectures/architectures/images/custom%20training/b_workflow.png)|

**Vertex AI Training**

In Vertex AI Training you can run your training code as a job where you specify the compute resources to use. For tips on preparing code, running training jobs, and workflows for building custom containers with software and training code combined please visit these [tips notebooks](../Tips/readme.md) in this repository:
- [Python Packages](../Tips/Python%20Packages.ipynb)
- [Python Custom Containers](../Tips/Python%20Custom%20Containers.ipynb)
- [Python Training](../Tips/Python%20Training.ipynb)
- [Python Job Parameters](../Tips/Python%20Job%20Parameters.ipybnd)

<p align="center" width="100%">
    <img src="../architectures/overview/training.png" width="45%">
    &nbsp; &nbsp; &nbsp; &nbsp;
    <img src="../architectures/overview/training2.png" width="45%">
</p>

**Prerequisites:**
-  [01 - BigQuery - Table Data Source](../01%20-%20Data%20Sources/01%20-%20BigQuery%20-%20Table%20Data%20Source.ipynb)
-  Understanding:
    -  Model overview in [05 - Vertex AI Custom Model - TensorFlow - in Notebook](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20in%20Notebook.ipynb)
    -  Convert notebook code to Python Script in [05 - Vertex AI Custom Model - TensorFlow - Notebook to Script](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Notebook%20to%20Script.ipynb)

**Resources:**
-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  [Create a Python source distribution](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container) for a Vertex AI custom training job
    - More on building custom packages [here](https://packaging.python.org/en/latest/tutorials/packaging-projects/)
    - A tips notebooks covering multiple ways to build a package: [Python Packages](../Tips/Python%20Packages.ipynb)
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
   -  [List](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)

<!--
**Conceptual Flow & Workflow**

<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/05b_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/05b_console.png" width="45%">
</p>
-->

---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/05%20-%20TensorFlow/05b%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Custom%20Job%20With%20Python%20Source%20Distribution.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('kfp', 'kfp'),
    ('google_cloud_pipeline_components', 'google-cloud-pipeline-components'),
    ('build', 'build')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [4]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [5]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [6]:
REGION = 'us-central1'
EXPERIMENT = '05b'
SERIES = '05'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# specify a GCS Bucket
GCS_BUCKET = PROJECT_ID

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id,splits' # add more variables to the string with comma delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [7]:
from google.cloud import aiplatform
from datetime import datetime
import importlib
from IPython.display import Markdown as md
from google.cloud import bigquery
from google.cloud import storage
#from google.protobuf import json_format
#from google.protobuf.struct_pb2 import Value
import json
import os, shutil
import numpy as np
import pandas as pd

clients:

In [8]:
aiplatform.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

parameters:

In [9]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
URI = f"gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}"
DIR = f"temp/{EXPERIMENT}"

In [10]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

List the service accounts current roles:

In [11]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/bigquery.admin
roles/owner
roles/run.admin
roles/secretmanager.secretAccessor
roles/storage.objectAdmin


>Note: If the resulting list is missing [roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles) then [revisit the setup notebook](../00%20-%20Setup/00%20-%20Environment%20Setup.ipynb#permissions) and add this permission to the service account with the provided instructions.

environment:

In [12]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

Experiment Tracking:

In [13]:
FRAMEWORK = 'tf'
TASK = 'classification'
MODEL_TYPE = 'dnn'
EXPERIMENT_NAME = f'experiment-{SERIES}-{EXPERIMENT}-{FRAMEWORK}-{TASK}-{MODEL_TYPE}'
RUN_NAME = f'run-{TIMESTAMP}'

---
## Get Vertex AI Experiments Tensorboard Instance Name
[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) has managed [Tensorboard](https://www.tensorflow.org/tensorboard) instances that you can track Tensorboard Experiments (a training run or hyperparameter tuning sweep).  

The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

This code checks to see if a Tensorboard Instance has been created in the project, retrieves it if so, creates it otherwise:

In [14]:
tb = aiplatform.Tensorboard.list(filter=f"labels.series={SERIES}")
if tb:
    tb = tb[0]
else: 
    tb = aiplatform.Tensorboard.create(display_name = SERIES, labels = {'series' : f'{SERIES}'})

In [15]:
tb.resource_name

'projects/1026793852137/locations/us-central1/tensorboards/7876136041294331904'

---
## Setup Vertex AI Experiments

The code in this section initializes the experiment and starts a run that represents this notebook.  Throughout the notebook sections for model training and evaluation information will be logged to the experiment using:
- [.log_params](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_params)
- [.log_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_metrics)
- [.log_time_series_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_time_series_metrics)

In [16]:
aiplatform.init(experiment = EXPERIMENT_NAME, experiment_tensorboard = tb.resource_name)

---
## Training

### Python File for Training

This notebook trains the same Tensorflow Keras model from [05 - Vertex AI Custom Model - TensorFlow - in Notebook](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20in%20Notebook.ipynb) by first modifying and saving the training code to a python script as shown in [05 - Vertex AI Custom Model - TensorFlow - Notebook to Script](./05%20-%20Vertex%20AI%20Custom%20Model%20-%20TensorFlow%20-%20Notebook%20to%20Script.ipynb) which stores the script in [`./code/train.py`](./code/train.py).

Retrieve the training script (if not already included in a clone of this repository):

In [17]:
file = "./code/train.py"
if not os.path.exists(file):
    print('Retrieving document...')
    if not os.path.exists(os.path.dirname(file)):
      os.makedirs(os.path.dirname(file))
    import requests, urllib.parse
    r = requests.get(f'https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/05%20-%20TensorFlow/{urllib.parse.quote(file[2:])}')
    open(file, 'wb').write(r.content)
    print(f'Document now at `{file}`')
else:
    print(f'Document Found at `{file}`')

Document Found at `./code/train.py`


**Review the script:**

In [18]:
SCRIPT_PATH = './code/train.py'

with open(SCRIPT_PATH, 'r') as file:
    data = file.read()
md(f"```python\n\n{data}\n```")

```python


# package import
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf
from google.cloud import bigquery
from google.cloud import aiplatform
import argparse
import os

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--epochs', dest = 'epochs', default = 10, type = int, help = 'Number of Epochs')
parser.add_argument('--batch_size', dest = 'batch_size', default = 32, type = int, help = 'Batch Size')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str)#, nargs='*')
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--bq_project', dest = 'bq_project', type=str)
parser.add_argument('--bq_dataset', dest = 'bq_dataset', type=str)
parser.add_argument('--bq_table', dest = 'bq_table', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--experiment', dest = 'experiment', type=str)
parser.add_argument('--series', dest = 'series', type=str)
parser.add_argument('--experiment_name', dest = 'experiment_name', type=str)
parser.add_argument('--run_name', dest = 'run_name', type=str)
args = parser.parse_args()

# clients
bq = bigquery.Client(project = args.project_id)
aiplatform.init(project = args.project_id, location = args.region)

# Vertex AI Experiment
if args.run_name in [run.name for run in aiplatform.ExperimentRun.list(experiment = args.experiment_name)]:
    expRun = aiplatform.ExperimentRun(run_name = args.run_name, experiment = args.experiment_name)
else:
    expRun = aiplatform.ExperimentRun.create(run_name = args.run_name, experiment = args.experiment_name)
expRun.log_params({'experiment': args.experiment, 'series': args.series, 'project_id': args.project_id})

# get schema from bigquery source
query = f"SELECT * FROM {args.bq_project}.{args.bq_dataset}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{args.bq_table}'"
schema = bq.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bq.query(query = f'SELECT DISTINCT {args.var_target} FROM {args.bq_project}.{args.bq_dataset}.{args.bq_table} WHERE {args.var_target} is not null').to_dataframe()
nclasses = nclasses.shape[0]
expRun.log_params({'data_source': f'bq://{args.bq_project}.{args.bq_dataset}.{args.bq_table}', 'nclasses': nclasses, 'var_split': 'splits', 'var_target': args.var_target})

# Make a list of columns to omit
OMIT = [x for x in args.var_omit.split(',') if x != '']

# use schema to prepare a list of columns to read from BigQuery
selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

# all the columns in this data source are either float64 or int64
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in schema[~schema.column_name.isin(OMIT)].data_type.tolist()]

# remap input data to Tensorflow inputs of features and target
def transTable(row_dict):
    target = row_dict.pop(args.var_target)
    target = tf.one_hot(tf.cast(target, tf.int64), nclasses)
    target = tf.cast(target, tf.float32)
    return(row_dict, target)

# function to setup a bigquery reader with Tensorflow I/O
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{args.project_id}",
        project_id = args.bq_project,
        table_id = args.bq_table,
        dataset_id = args.bq_dataset,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

# setup feed for train, validate and test
train = bq_reader('TRAIN').parallel_read_rows().prefetch(1).map(transTable).shuffle(args.batch_size*10).batch(args.batch_size)
validate = bq_reader('VALIDATE').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)
test = bq_reader('TEST').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)
expRun.log_params({'training.batch_size': args.batch_size, 'training.shuffle': 10*args.batch_size, 'training.prefetch': 1})
# Logistic Regression

# feature list
numeric_features = [feature for feature in schema[~schema.column_name.isin(OMIT + [args.var_target])]['column_name'].to_list()]

# feature inputs
features = [tf.keras.Input(shape = (1,), dtype = dtypes.float64, name = feature) for feature in numeric_features]

# normalize features - before training
#normalized_features = []
#for feature in features:
#    normalizer = tf.keras.layers.Normalization(axis = None, name = feature.name + '_normalized')
#    feature_data = train.map(lambda x, y: x[feature.name])
#    normalizer.adapt(feature_data)
#    normalized_features.append(normalizer(feature))

# concatenate features
all_features = tf.keras.layers.Concatenate(name = 'feature_layer')(features)
#all_features = tf.keras.layers.Concatenate(name = 'feature_layer')(normalized_features) # (features)

# batch normalization of inputs - during training
all_features = tf.keras.layers.BatchNormalization(name = 'batch_normalization_layer')(all_features)

# logistic - using softmax activation to nclasses
logistic = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax, name = 'logistic')(all_features)

# the model
model = tf.keras.Model(
    inputs = features,
    outputs = logistic,
    name = args.experiment
)

# compile the model
model.compile(
    optimizer = tf.keras.optimizers.SGD(), #SGD or Adam
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy', tf.keras.metrics.AUC(curve = 'PR', name = 'auprc')]
)

# setup tensorboard logs and train
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'], histogram_freq=1)
history = model.fit(train, epochs = args.epochs, callbacks = [tensorboard_callback], validation_data = validate)
expRun.log_params({'training.epochs': history.params['epochs']})
for e in range(0, history.params['epochs']):
    expRun.log_time_series_metrics(
        {
            'train_loss': history.history['loss'][e],
            'train_accuracy': history.history['accuracy'][e],
            'train_auprc': history.history['auprc'][e],
            'val_loss': history.history['val_loss'][e],
            'val_accuracy': history.history['val_accuracy'][e],
            'val_auprc': history.history['val_auprc'][e]
        }
    )

# test evaluations:
loss, accuracy, auprc = model.evaluate(test)
expRun.log_metrics({'test_loss': loss, 'test_accuracy': accuracy, 'test_auprc': auprc})

# val evaluations:
loss, accuracy, auprc = model.evaluate(validate)
expRun.log_metrics({'val_loss': loss, 'val_accuracy': accuracy, 'val_auprc': auprc})

# training evaluations:
loss, accuracy, auprc = model.evaluate(train)
expRun.log_metrics({'train_loss': loss, 'train_accuracy': accuracy, 'train_auprc': auprc})

# output the model save files
model.save(os.getenv("AIP_MODEL_DIR"))
expRun.log_params({'model.save': os.getenv("AIP_MODEL_DIR")})
expRun.end_run()

```

### Construct Python Package

More details [here](https://packaging.python.org/en/latest/tutorials/packaging-projects/) and a tips notebook [here](../Tips/Python%20Packages.ipynb).

In [19]:
DIR

'temp/05b'

In [20]:
os.listdir(DIR)

['request.json', '05b_trainer']

#### Create the folder structure

In [21]:
path = DIR + f'/{EXPERIMENT}_trainer/src/{EXPERIMENT}_trainer'

if not os.path.exists(path):
    os.makedirs(path)

In [22]:
for root, dirs, files in os.walk(DIR):
    print(root)

temp/05b
temp/05b/05b_trainer
temp/05b/05b_trainer/src
temp/05b/05b_trainer/src/05b_trainer.egg-info
temp/05b/05b_trainer/src/05b_trainer
temp/05b/05b_trainer/dist


#### Add files to directory
Copy the `./code/train.py` file

In [23]:
shutil.copyfile(SCRIPT_PATH, f'{DIR}/{EXPERIMENT}_trainer/src/{EXPERIMENT}_trainer/train.py')

'temp/05b/05b_trainer/src/05b_trainer/train.py'

Create an empty `__init__.py` file

In [24]:
with open(f'{DIR}/{EXPERIMENT}_trainer/src/{EXPERIMENT}_trainer/__init__.py', 'w') as file: pass

Create a toml file for building the package:

In [87]:
with open(f'{DIR}/{EXPERIMENT}_trainer/pyproject.toml', 'w') as file:
    file.write(f"""[build-system]
requires = ["setuptools"]
build-backend = "setuptools.build_meta"

[project]
name = '{EXPERIMENT}_trainer'
version = '0.1'
dependencies = ['tensorflow_io', 'google-cloud-aiplatform>={aiplatform.__version__}', 'db-dtypes', 'protobuf>={importlib.metadata.version('protobuf')}']
description = 'Training Package'
authors = [{{name = 'statmike'}}]
""")

list directory:

In [26]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        print(os.path.join(root, f))

temp/05b/request.json
temp/05b/05b_trainer/pyproject.toml
temp/05b/05b_trainer/src/05b_trainer.egg-info/top_level.txt
temp/05b/05b_trainer/src/05b_trainer.egg-info/PKG-INFO
temp/05b/05b_trainer/src/05b_trainer.egg-info/requires.txt
temp/05b/05b_trainer/src/05b_trainer.egg-info/dependency_links.txt
temp/05b/05b_trainer/src/05b_trainer.egg-info/SOURCES.txt
temp/05b/05b_trainer/src/05b_trainer/__init__.py
temp/05b/05b_trainer/src/05b_trainer/train.py
temp/05b/05b_trainer/dist/05b_trainer-0.1.tar.gz
temp/05b/05b_trainer/dist/05b_trainer-0.1-py3-none-any.whl


#### Build the Python distribution archives:

The build process creates both a `.tar.gz` source distribution and a `.whl` built distribution

In [27]:
!cd ./{DIR}/{EXPERIMENT}_trainer && python -m build

* Creating virtualenv isolated environment...
* Installing packages in isolated environment... (setuptools)
* Getting build dependencies for sdist...
running egg_info
writing src/05b_trainer.egg-info/PKG-INFO
writing dependency_links to src/05b_trainer.egg-info/dependency_links.txt
writing requirements to src/05b_trainer.egg-info/requires.txt
writing top-level names to src/05b_trainer.egg-info/top_level.txt
reading manifest file 'src/05b_trainer.egg-info/SOURCES.txt'
writing manifest file 'src/05b_trainer.egg-info/SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing src/05b_trainer.egg-info/PKG-INFO
writing dependency_links to src/05b_trainer.egg-info/dependency_links.txt
writing requirements to src/05b_trainer.egg-info/requires.txt
writing top-level names to src/05b_trainer.egg-info/top_level.txt
reading manifest file 'src/05b_trainer.egg-info/SOURCES.txt'
writing manifest file 'src/05b_trainer.egg-info/SOURCES.txt'

running check
creating 05b_trainer-0.1
creating 0

list directory:

In [28]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        print(os.path.join(root, f))

temp/05b/request.json
temp/05b/05b_trainer/pyproject.toml
temp/05b/05b_trainer/src/05b_trainer.egg-info/top_level.txt
temp/05b/05b_trainer/src/05b_trainer.egg-info/PKG-INFO
temp/05b/05b_trainer/src/05b_trainer.egg-info/requires.txt
temp/05b/05b_trainer/src/05b_trainer.egg-info/dependency_links.txt
temp/05b/05b_trainer/src/05b_trainer.egg-info/SOURCES.txt
temp/05b/05b_trainer/src/05b_trainer/__init__.py
temp/05b/05b_trainer/src/05b_trainer/train.py
temp/05b/05b_trainer/dist/05b_trainer-0.1.tar.gz
temp/05b/05b_trainer/dist/05b_trainer-0.1-py3-none-any.whl


#### Copy to GCS

Here the folder structure for DIR will be copied to the GCS Bucket used across this project.  This section uses skills that are discussed in more detail in the [Python Client for GCS](../Tips/Python%20Client%20for%20GCS.ipynb) notebook.

Get the bucket:

In [29]:
bucket = gcs.lookup_bucket(GCS_BUCKET)

list files to upload:

In [30]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        print(os.path.join(root, f)[len(DIR):])

/request.json
/05b_trainer/pyproject.toml
/05b_trainer/src/05b_trainer.egg-info/top_level.txt
/05b_trainer/src/05b_trainer.egg-info/PKG-INFO
/05b_trainer/src/05b_trainer.egg-info/requires.txt
/05b_trainer/src/05b_trainer.egg-info/dependency_links.txt
/05b_trainer/src/05b_trainer.egg-info/SOURCES.txt
/05b_trainer/src/05b_trainer/__init__.py
/05b_trainer/src/05b_trainer/train.py
/05b_trainer/dist/05b_trainer-0.1.tar.gz
/05b_trainer/dist/05b_trainer-0.1-py3-none-any.whl


list of desired bucket object URIs:

In [31]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        filepath = os.path.join(root, f)
        gcspath = f'{SERIES}/{EXPERIMENT}/trainer/{filepath[len(DIR)+1:]}'
        print(gcspath)

05/05b/trainer/request.json
05/05b/trainer/05b_trainer/pyproject.toml
05/05b/trainer/05b_trainer/src/05b_trainer.egg-info/top_level.txt
05/05b/trainer/05b_trainer/src/05b_trainer.egg-info/PKG-INFO
05/05b/trainer/05b_trainer/src/05b_trainer.egg-info/requires.txt
05/05b/trainer/05b_trainer/src/05b_trainer.egg-info/dependency_links.txt
05/05b/trainer/05b_trainer/src/05b_trainer.egg-info/SOURCES.txt
05/05b/trainer/05b_trainer/src/05b_trainer/__init__.py
05/05b/trainer/05b_trainer/src/05b_trainer/train.py
05/05b/trainer/05b_trainer/dist/05b_trainer-0.1.tar.gz
05/05b/trainer/05b_trainer/dist/05b_trainer-0.1-py3-none-any.whl


upload files as objects:

In [32]:
for root, dirs, files in os.walk(DIR):
    for f in files:
        filepath = os.path.join(root, f)
        gcspath = f'{SERIES}/{EXPERIMENT}/trainer/{filepath[len(DIR)+1:]}'
        blob = bucket.blob(gcspath)
        blob.upload_from_filename(filepath)

In [33]:
print(f"View the bucket directly here:\nhttps://console.cloud.google.com/storage/browser/{GCS_BUCKET}/{SERIES}/{EXPERIMENT};tab=objects&project={PROJECT_ID}")

View the bucket directly here:
https://console.cloud.google.com/storage/browser/statmike-mlops-349915/05/05b;tab=objects&project=statmike-mlops-349915


list files in bucket:

In [34]:
for blob in list(bucket.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}/trainer/')):
    print(blob.name)

05/05b/trainer/05b_trainer/dist/05b_trainer-0.1-py3-none-any.whl
05/05b/trainer/05b_trainer/dist/05b_trainer-0.1.tar.gz
05/05b/trainer/05b_trainer/pyproject.toml
05/05b/trainer/05b_trainer/src/05b_trainer.egg-info/PKG-INFO
05/05b/trainer/05b_trainer/src/05b_trainer.egg-info/SOURCES.txt
05/05b/trainer/05b_trainer/src/05b_trainer.egg-info/dependency_links.txt
05/05b/trainer/05b_trainer/src/05b_trainer.egg-info/requires.txt
05/05b/trainer/05b_trainer/src/05b_trainer.egg-info/top_level.txt
05/05b/trainer/05b_trainer/src/05b_trainer/__init__.py
05/05b/trainer/05b_trainer/src/05b_trainer/train.py
05/05b/trainer/request.json


get a list of source distributions:

In [35]:
SOURCES = [f'gs://{GCS_BUCKET}/{blob.name}' for blob in list(bucket.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}/trainer/')) if blob.name[-7:] == '.tar.gz']
SOURCES

['gs://statmike-mlops-349915/05/05b/trainer/05b_trainer/dist/05b_trainer-0.1.tar.gz']

### Choose Computing Environment

When using [custom training on Vertex AI](https://cloud.google.com/vertex-ai/docs/training/custom-training-methods) the compute environment is specified as parameters.  At a minimum this will include the [compute resources](https://cloud.google.com/vertex-ai/docs/training/configure-compute) and [container](https://cloud.google.com/vertex-ai/docs/training/configure-container-settings) URIs.

This example uses minimal compute with a single node and no accelerators (GPU).

For a container, a [pre-built custome training container](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for TensorFlow is being used and additional packages are being specified in the jobs parameters. This leads to two important considerations:
- What is the Python version on the container? 
    - This will have an impact when additional packages are being installed and the packages may have version requirments.
- What is the framework version on the container?
    - Make sure the options you need from a framework are included in the version you pick.

In [36]:
# Resources
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-12.py310:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-12:latest'

### Setup Training Job

In [37]:
CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--bq_project=" + BQ_PROJECT,
    "--bq_dataset=" + BQ_DATASET,
    "--bq_table=" + BQ_TABLE,
    "--region=" + REGION,
    "--experiment=" + EXPERIMENT,
    "--series=" + SERIES,
    "--experiment_name=" + EXPERIMENT_NAME,
    "--run_name=" + RUN_NAME
]

In [38]:
MACHINE_SPEC = {
    "machine_type": TRAIN_COMPUTE,
    "accelerator_count": 0
}

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": MACHINE_SPEC,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": SOURCES,
            "python_module": f"{EXPERIMENT}_trainer.train",
            "args": CMDARGS
        }
    }
]

In [39]:
customJob = aiplatform.CustomJob(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    worker_pool_specs = WORKER_POOL_SPEC,
    base_output_dir = f"{URI}/models/{TIMESTAMP}",
    staging_bucket = f"{URI}/models/{TIMESTAMP}",
    labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
)

### Run Training Job

In [40]:
customJob.run(
    service_account = SERVICE_ACCOUNT,
    tensorboard = tb.resource_name
)

Creating CustomJob
CustomJob created. Resource name: projects/1026793852137/locations/us-central1/customJobs/4761853674416766976
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/1026793852137/locations/us-central1/customJobs/4761853674416766976')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4761853674416766976?project=1026793852137
View Tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7876136041294331904+experiments+4761853674416766976
CustomJob projects/1026793852137/locations/us-central1/customJobs/4761853674416766976 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1026793852137/locations/us-central1/customJobs/4761853674416766976 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1026793852137/locations/us-central1/customJobs/4761853674416766976 current state:
JobState.JOB_STATE_PENDING


In [41]:
customJob.display_name

'05_05b_20231221183939'

In [42]:
customJob.resource_name

'projects/1026793852137/locations/us-central1/customJobs/4761853674416766976'

Create hyperlinks to job and tensorboard here:

In [43]:
job_link = f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/training/{customJob.resource_name.split('/')[-1]}/cpu?cloudshell=false&project={PROJECT_ID}"
board_link = f"https://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{customJob.resource_name.split('/')[-1]}"

print(f'Review the Custom Job here:\n{job_link}')
print(f'Review the TensorBoard From the Job here:\n{board_link}')

Review the Custom Job here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/training/4761853674416766976/cpu?cloudshell=false&project=statmike-mlops-349915
Review the TensorBoard From the Job here:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7876136041294331904+experiments+4761853674416766976


---
## Serving

### Upload The Model

In [44]:
modelmatch = aiplatform.Model.list(filter = f'display_name={SERIES}_{EXPERIMENT} AND labels.series={SERIES} AND labels.experiment={EXPERIMENT}')

upload_model = True
if modelmatch:
    print("Model Already in Registry:")
    if RUN_NAME in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        upload_model = False
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        parent_model = modelmatch[0].resource_name

else:
    print('This is a new model, creating in model registry')
    parent_model = ''

if upload_model:
    model = aiplatform.Model.upload(
        display_name = f'{SERIES}_{EXPERIMENT}',
        model_id = f'model_{SERIES}_{EXPERIMENT}',
        parent_model =  parent_model,
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/models/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [RUN_NAME],
        version_description = RUN_NAME,
        labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}        
    )

Model Already in Registry:
Loading model as new default version.
Creating Model
Create Model backing LRO: projects/1026793852137/locations/us-central1/models/model_05_05b/operations/9147533153609449472
Model created. Resource name: projects/1026793852137/locations/us-central1/models/model_05_05b@8
To use this Model in another session:
model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/model_05_05b@8')


>**Note** on Version Aliases:
>Expectation is a name starting with `a-z` that can include `[a-zA-Z0-9-]`
>
>**Retrieve a Model Resource**
>[aiplatform.Model()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model)
>```Python
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}') # retrieves default version
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}@time-{TIMESTAMP}') # retrieves specific version
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}', version = f'time-{TIMESTAMP}') # retrieves specific version
```

In [45]:
print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}?project={PROJECT_ID}')

Review the model in the Vertex AI Model Registry:
https://console.cloud.google.com/vertex-ai/locations/us-central1/models/model_05_05b?project=statmike-mlops-349915


### Vertex AI Experiment Update and Review

In [46]:
expRun = aiplatform.ExperimentRun(run_name = RUN_NAME, experiment = EXPERIMENT_NAME)

In [47]:
expRun.log_params({
    'model.uri': model.uri,
    'model.display_name': model.display_name,
    'model.name': model.name,
    'model.resource_name': model.resource_name,
    'model.version_id': model.version_id,
    'model.versioned_resource_name': model.versioned_resource_name,
    'customJobs.display_name': customJob.display_name,
    'customJobs.resource_name': customJob.resource_name,
    'customJobs.link': job_link,
    'customJobs.tensorboard': board_link
})

Complete the experiment run:

In [48]:
expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

Retrieve the experiment:

In [49]:
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)

In [50]:
exp.get_data_frame()

,experiment_name,run_name,run_type,state,param.training.epochs,param.customJobs.resource_name,param.training.shuffle,param.model.version_id,param.customJobs.tensorboard,param.project_id,...,metric.train_loss,metric.val_loss,metric.train_auprc,metric.val_auprc,time_series_metric.val_accuracy,time_series_metric.train_auprc,time_series_metric.train_loss,time_series_metric.val_auprc,time_series_metric.train_accuracy,time_series_metric.val_loss
0,experiment-05-05b-tf-classification-dnn,run-20231221183939,system.ExperimentRun,COMPLETE,10.0,projects/1026793852137/locations/us-central1/c...,1000.0,8,https://us-central1.tensorboard.googleusercont...,statmike-mlops-349915,...,0.007657,0.007836,0.999277,0.999517,0.999079,0.999508,0.005285,0.999517,0.999154,0.007836
1,experiment-05-05b-tf-classification-dnn,run-20231221020002,system.ExperimentRun,COMPLETE,10.0,projects/1026793852137/locations/us-central1/c...,1000.0,7,https://us-central1.tensorboard.googleusercont...,statmike-mlops-349915,...,0.006960,0.006584,0.999342,0.999384,0.999044,0.999408,0.005250,0.999384,0.999228,0.006584
2,experiment-05-05b-tf-classification-dnn,run-20231221012719,system.ExperimentRun,COMPLETE,10.0,projects/1026793852137/locations/us-central1/c...,1000.0,6,https://us-central1.tensorboard.googleusercont...,statmike-mlops-349915,...,0.007424,0.006949,0.999341,0.999428,0.999079,0.999549,0.004473,0.999428,0.999224,0.006949
3,experiment-05-05b-tf-classification-dnn,run-20231220154420,system.ExperimentRun,COMPLETE,10.0,projects/1026793852137/locations/us-central1/c...,1000.0,5,https://us-central1.tensorboard.googleusercont...,statmike-mlops-349915,...,0.008338,0.007728,0.999376,0.999439,0.999044,0.999496,0.005402,0.999439,0.999241,0.007728
4,experiment-05-05b-tf-classification-dnn,run-20230210130602,system.ExperimentRun,COMPLETE,10.0,projects/1026793852137/locations/us-central1/c...,1000.0,4,https://us-central1.tensorboard.googleusercont...,statmike-mlops-349915,...,0.007354,0.006656,0.999341,0.999519,0.999044,0.999513,0.004860,0.999519,0.999171,0.006656


Review the Experiments TensorBoard to compare runs:

In [51]:
print(f"The Experiment TensorBoard Link:\nhttps://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{exp.name}")

The Experiment TensorBoard Link:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7876136041294331904+experiments+experiment-05-05b-tf-classification-dnn


In [52]:
expRun.get_time_series_data_frame()

,step,wall_time,val_accuracy,train_auprc,train_loss,val_auprc,train_accuracy,val_loss
0,1,2023-12-21 18:49:48.632000+00:00,0.997805,0.993710,0.076729,0.998958,0.979282,0.020381
1,2,2023-12-21 18:49:48.744000+00:00,0.998548,0.999289,0.011543,0.999410,0.998180,0.012556
2,3,2023-12-21 18:49:48.840000+00:00,0.998761,0.999436,0.008163,0.999463,0.998728,0.010461
3,4,2023-12-21 18:49:48.924000+00:00,0.998867,0.999444,0.006926,0.999527,0.998891,0.009599
4,5,2023-12-21 18:49:49.030000+00:00,0.998902,0.999441,0.006431,0.999448,0.998961,0.009045
5,6,2023-12-21 18:49:49.126000+00:00,0.998938,0.999483,0.005762,0.999457,0.999035,0.008637
6,7,2023-12-21 18:49:49.228000+00:00,0.998938,0.999447,0.005806,0.999508,0.999053,0.008362
7,8,2023-12-21 18:49:49.328000+00:00,0.999044,0.999456,0.005647,0.999512,0.999119,0.008210
8,9,2023-12-21 18:49:49.436000+00:00,0.999079,0.999476,0.005430,0.999515,0.999101,0.007983
9,10,2023-12-21 18:49:49.527000+00:00,0.999079,0.999508,0.005285,0.999517,0.999154,0.007836


### Review Experiment and Run in Console

In [53]:
print(f'Review The Experiment in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/experiments/{EXPERIMENT_NAME}?project={PROJECT_ID}')

Review The Experiment in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/experiments/experiment-05-05b-tf-classification-dnn?project=statmike-mlops-349915


In [54]:
print(f'Review The Experiment Run in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/experiments/{EXPERIMENT_NAME}/runs/{EXPERIMENT_NAME}-{RUN_NAME}?project={PROJECT_ID}')

Review The Experiment Run in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/experiments/experiment-05-05b-tf-classification-dnn/runs/experiment-05-05b-tf-classification-dnn-run-20231221183939?project=statmike-mlops-349915


### Compare This Run Using Experiments

Get a list of all experiments in this project:

In [55]:
experiments = aiplatform.Experiment.list()

Remove experiments not in the SERIES:

In [56]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

Combine the runs from all experiments in SERIES into a single dataframe:

In [57]:
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
results = pd.concat(results)

experiment-05-05f-tf-classification-dnn
experiment-05-05i-tf-classification-dnn
experiment-05-05h-tf-classification-dnn
experiment-05-05g-tf-classification-dnn
experiment-05-05e-tf-classification-dnn
experiment-05-05d-tf-classification-dnn
experiment-05-05c-tf-classification-dnn
experiment-05-05b-tf-classification-dnn
experiment-05-05a-tf-classification-dnn
experiment-05-05-tf-classification-dnn


Create ranks for models within experiment and across the entire SERIES:

In [58]:
def ranker(metric = 'metric.test_auprc'):
    ranks = results[['experiment_name', 'run_name', 'param.model.display_name', 'param.model.version_id', metric]].copy().reset_index(drop = True)
    ranks['series_rank'] = ranks[metric].rank(method = 'dense', ascending = False)
    ranks['experiment_rank'] = ranks.groupby('experiment_name')[metric].rank(method = 'dense', ascending = False)
    return ranks.sort_values(by = ['experiment_name', 'run_name'])
    
ranks = ranker('metric.test_auprc')
ranks

,experiment_name,run_name,param.model.display_name,param.model.version_id,metric.test_auprc,series_rank,experiment_rank
157,experiment-05-05-tf-classification-dnn,run-20230210115433,05_05,6,0.999130,131.0,16.0
156,experiment-05-05-tf-classification-dnn,run-20230213133609,NaN,NaN,0.999309,114.0,11.0
155,experiment-05-05-tf-classification-dnn,run-20230308225745,05_05,7,0.999469,77.0,4.0
154,experiment-05-05-tf-classification-dnn,run-20230324103811,05_05,8,0.999299,116.0,12.0
153,experiment-05-05-tf-classification-dnn,run-20230324104933,NaN,NaN,0.999469,78.0,5.0
...,...,...,...,...,...,...,...
59,experiment-05-05i-tf-classification-dnn,run-20230211221928-5,NaN,NaN,0.999584,13.0,7.0
58,experiment-05-05i-tf-classification-dnn,run-20230211221928-6,NaN,NaN,0.999538,36.0,13.0
57,experiment-05-05i-tf-classification-dnn,run-20230211221928-7,NaN,NaN,0.999538,37.0,14.0
56,experiment-05-05i-tf-classification-dnn,run-20230211221928-8,NaN,NaN,0.999584,8.0,3.0


In [59]:
current_rank = ranks.loc[(ranks['param.model.display_name'] == model.display_name) & (ranks['param.model.version_id'] == model.version_id)]
current_rank

,experiment_name,run_name,param.model.display_name,param.model.version_id,metric.test_auprc,series_rank,experiment_rank
99,experiment-05-05b-tf-classification-dnn,run-20231221183939,05_05b,8,0.999337,113.0,4.0


In [60]:
print(f"The current model is ranked {current_rank['experiment_rank'].iloc[0]} within this experiment and {current_rank['series_rank'].iloc[0]} across this series.")

The current model is ranked 4.0 within this experiment and 113.0 across this series.


### Create/Retrieve The Endpoint For This Series

In [61]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES} AND display_name={SERIES}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}",
        labels = {'series' : f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")
    
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

Endpoint Exists: projects/1026793852137/locations/us-central1/endpoints/725723853820526592
Review the Endpoint in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/725723853820526592?project=statmike-mlops-349915


In [62]:
endpoint.display_name

'05'

In [63]:
endpoint.traffic_split

{'2423682068408958976': 100}

In [64]:
deployed_models = endpoint.list_models()
#deployed_models

### Should This Model Be Deployed?
Is it better than the model already deployed on the endpoint?

In [65]:
deploy = False
if deployed_models:
    for deployed_model in deployed_models:
        deployed_rank = ranks.loc[(ranks['param.model.display_name'] == deployed_model.display_name) & (ranks['param.model.version_id'] == deployed_model.model_version_id)]['series_rank'].iloc[0]
        model_rank = current_rank['series_rank'].iloc[0]
        if deployed_model.display_name == model.display_name and deployed_model.model_version_id == model.version_id:
            print(f'The current model/version is already deployed.')
            break
        elif model_rank <= deployed_rank:
            deploy = True
            print(f'The current model is ranked better ({model_rank}) than a currently deployed model ({deployed_rank}).')
            break
    if deploy == False: print(f'The current model is ranked worse ({model_rank}) than a currently deployed model ({deployed_rank})')
else: 
    deploy = True
    print('No models currently deployed.')

The current model is ranked worse (113.0) than a currently deployed model (3.0)


### Deploy Model To Endpoint

In [66]:
if deploy:
    print(f'Deploying model with 100% of traffic...')
    endpoint.deploy(
        model = model,
        deployed_model_display_name = model.display_name,
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    )
else: print(f'Not deploying - current model is worse ({model_rank}) than the currently deployed model ({deployed_rank})')

Not deploying - current model is worse (113.0) than the currently deployed model (3.0)


### Remove Deployed Models without Traffic

In [67]:
for deployed_model in endpoint.list_models():
    if deployed_model.id in endpoint.traffic_split:
        print(f"Model {deployed_model.display_name} with version {deployed_model.model_version_id} has traffic = {endpoint.traffic_split[deployed_model.id]}")
    else:
        endpoint.undeploy(deployed_model_id = deployed_model.id)
        print(f"Undeploying {deployed_model.display_name} with version {deployed_model.model_version_id} because it has no traffic.")

Model 05_05 with version 18 has traffic = 100


In [68]:
endpoint.traffic_split

{'2423682068408958976': 100}

In [69]:
#endpoint.list_models()

---
## Prediction

See many more details on requesting predictions in the [05Tools - Prediction](./05Tools%20-%20Prediction.ipynb) notebook.

### Prepare a record for prediction: instance and parameters lists

In [70]:
n = 10
# Make a list of columns to omit
OMIT = [x for x in VAR_OMIT.split(',') if x != '']
pred = bq.query(
    query = f"""
        SELECT * EXCEPT({','.join([VAR_TARGET] + OMIT)})
        FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}
        WHERE splits='TEST'
        LIMIT {n}
        """
).to_dataframe()

In [71]:
pred

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.240428,0.037603,0.380026,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.265080,-0.060003,-0.053585,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,-0.374356,0.196006,0.656552,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-0.923275,-0.545992,-0.252324,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0
4,165236,2.180149,0.218732,-2.637726,0.348776,1.063546,-1.249197,0.942021,-0.547652,-0.087823,...,-0.250653,0.234502,0.825237,-0.176957,0.563779,0.730183,0.707494,-0.131066,-0.090428,0.0
5,62606,1.199408,0.352007,0.379645,1.372017,0.291347,0.524919,-0.117555,0.132907,-0.935169,...,-0.042979,-0.050291,-0.126609,-0.022218,-0.599026,0.258188,0.928721,-0.058988,-0.008856,0.0
6,90719,1.937447,0.337882,-0.000630,3.816486,0.276515,1.079842,-0.730626,0.197353,1.137566,...,-0.315667,-0.038376,0.208914,0.160189,-0.015145,-0.162678,-0.000843,-0.018178,-0.039339,0.0
7,113350,1.891900,0.401086,-0.119983,4.047500,0.049952,0.192793,-0.108512,-0.040400,-0.390391,...,-0.267639,0.094177,0.613712,0.070986,0.079543,0.135219,0.128961,0.003667,-0.045079,0.0
8,156499,0.060003,1.461355,0.378915,2.835455,1.626526,-0.164732,1.551858,-0.412927,-1.735264,...,-0.175275,0.042293,0.277536,-0.123379,1.081552,-0.053079,-0.149809,-0.314438,-0.216539,0.0
9,73902,-1.859260,2.158799,1.085671,2.615483,0.246660,2.133925,-1.569015,-2.612353,-1.312509,...,0.590142,-0.867178,-0.700479,0.231972,-1.374527,0.140285,0.128806,0.153606,0.092042,0.0


In [76]:
newobs = pred.to_dict(orient = 'records')
#newobs[0]

In [77]:
newobs = [{k:[v] for k,v in newob.items()} for newob in newobs]

In [78]:
newobs[0]

{'Time': [35337],
 'V1': [1.0928441854981998],
 'V2': [-0.0132303486713432],
 'V3': [1.35982868199426],
 'V4': [2.7315370965921004],
 'V5': [-0.707357349219652],
 'V6': [0.8738370029866129],
 'V7': [-0.7961301510622031],
 'V8': [0.437706509544851],
 'V9': [0.39676985012996396],
 'V10': [0.587438102569443],
 'V11': [-0.14979756231827498],
 'V12': [0.29514781622888103],
 'V13': [-1.30382621882143],
 'V14': [-0.31782283120234495],
 'V15': [-2.03673231037199],
 'V16': [0.376090905274179],
 'V17': [-0.30040350116459497],
 'V18': [0.433799615590844],
 'V19': [-0.145082264348681],
 'V20': [-0.240427548108996],
 'V21': [0.0376030733329398],
 'V22': [0.38002620963091405],
 'V23': [-0.16764742731151097],
 'V24': [0.0275573495476881],
 'V25': [0.59211469704354],
 'V26': [0.219695164116351],
 'V27': [0.0369695108704894],
 'V28': [0.010984441006191],
 'Amount': [0.0]}

In [79]:
#instances = [json_format.ParseDict(newobs[0], Value())]

### Get Predictions: Python Client

In [80]:
prediction = endpoint.predict(instances = newobs[0:1])
prediction

Prediction(predictions=[[0.999759495, 0.000240550202]], deployed_model_id='2423682068408958976', model_version_id='18', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05_05', explanations=None)

In [81]:
prediction = endpoint.predict(instances = newobs)
prediction

Prediction(predictions=[[0.999759436, 0.000240550187], [0.999389887, 0.000610154064], [0.999763489, 0.000236547974], [0.999877691, 0.000122421377], [0.999873161, 0.000126962157], [0.999579489, 0.000420496828], [0.999929368, 7.053015e-05], [0.999938548, 6.13750744e-05], [0.999892116, 0.00010785809], [0.999943733, 5.62981877e-05]], deployed_model_id='2423682068408958976', model_version_id='18', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05_05', explanations=None)

In [82]:
prediction.predictions[0]

[0.999759436, 0.000240550187]

In [83]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [84]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": newobs[0:1]}))

In [85]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999759495,
      0.000240550202
    ]
  ],
  "deployedModelId": "2423682068408958976",
  "model": "projects/1026793852137/locations/us-central1/models/model_05_05",
  "modelDisplayName": "05_05",
  "modelVersionId": "18"
}


### Get Predictions: gcloud (CLI)

In [86]:
!gcloud ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999759495, 0.000240550202]]


---
## Remove Resources
see notebook "99 - Cleanup"